In [1]:
from __future__ import division
import matplotlib.pyplot as plt
from IPython.display import Math, Latex, display
from IPython.display import display as print
#from IPython import Cell
import sympy as sp
from sympy import symbols, Function
import sympy.functions as sym
from sympy import pprint
from sympy import Symbol
import sympy.printing as printing
from sympy.parsing.sympy_parser import parse_expr

sp.init_printing(use_latex=True)
%config Cell.options_default.lineNumbers = False;
%gui qt5
from numpy import *
x, y, z, t = symbols('x y z t')
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
%matplotlib --list
%precision 18

Available matplotlib backends: ['gtk3', 'inline', 'pdf', 'svg', 'gtk', 'osx', 'nbagg', 'ps', 'agg', 'qt', 'tk', 'qt5', 'wx', 'widget', 'notebook', 'qt4', 'ipympl']


'%.18f'

In [2]:
import ipyparallel as ipp
c = ipp.Client()
lview  = c.load_balanced_view()
lview.block = True
CORES = len(c[:])
v = c[:]
c.ids

In [3]:
def setter(a):
    """only copy read-only arrays"""
    if not a.flags.writeable:
        a=a.copy()
        a[0]=1
        return a
A = np.zeros(2)
c[0].apply_sync(setter, A)

array([1., 0.])

In [4]:
v['A'] = A

In [14]:
from queue import Queue
import pyopencl as cl
from threading import Thread

class cl_process():
	def __init__(self, _prog):
		self.X = np.array([])
		self.Y = np.array([])
		self._prog = _prog

		self.ctx = cl.create_some_context()
		self.queue = cl.CommandQueue(self.ctx)
		self.mf = cl.mem_flags
		self.program = cl.Program(self.ctx, self._prog).build()

	def cl_init(self, X=np.array([]), Y=np.array([])):
		self.prg = self.program
		self.X = np.array(X)
		self.Y = np.array(Y)

	def cl_build(self):		
		self.X_g = cl.Buffer(self.ctx, self.mf.READ_ONLY | self.mf.COPY_HOST_PTR, hostbuf=self.X)
		self.Y_g = cl.Buffer(self.ctx, self.mf.READ_ONLY | self.mf.COPY_HOST_PTR, hostbuf=self.Y)
		
	
	def cl_execute(self):
		self.res_g = cl.Buffer(self.ctx, self.mf.WRITE_ONLY, self.X.nbytes)
		self.prg.sum(self.queue, self.X.shape, None, self.X_g, self.Y_g, self.res_g)
		
		self.res_np = np.empty_like(self.X)
		cl.enqueue_copy(self.queue, self.res_np, self.res_g)

	def cl_get_result(self):
		return self.res_np


In [15]:
# Our OpenCL_1.1 program
myprog = """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
"""
mycl = cl_process(myprog)

In [29]:
X = np.random.random(10000).astype(np.float32)
Y = np.random.random(10000).astype(np.float32)
mycl.cl_init(X,Y)
mycl.cl_build()
mycl.cl_execute()
a = mycl.cl_get_result()
a

array([1.0215262 , 1.6900655 , 1.1601907 , ..., 1.2818075 , 1.1829779 ,
       0.67918396], dtype=float32)

In [30]:
import clr
clr.AddReference("System.Windows.Forms")
clr.AddReference("System.Drawing")
from System.Windows.Forms import *
from System.Drawing import *

f = Form()
f.Size =  Size(800,600)
f.Text = "Source Image"
f1 = Form()
f1.Size =  Size(800,600)
f.Text = "Target Image"
editor = Form()
editor.Size =  Size(320,200)
editor.Text = "OpenCL Editor"
#font = Font("Verdana", 9)

b = Bitmap(800,600)
g = Graphics.FromImage(b);
g.Clear(Color.Green);
c = Color.FromArgb(255,0,0)

for a in range(len(X)):
	b.SetPixel(X[a]*800,Y[a]*600, c)

p = PictureBox()
p.Size = Size(b.Width, b.Height)
p.Visible = True
p.Location = Point(0,0)
p.Image = b
f.Controls.Add(p)
f.Update()

#def click(f, a):
#f.Click += click

f.Show()